In [1]:
# -*- coding: utf-8 -*-
# author: juudit.ottelin@ntnu.no
# date: 16.4.2024
#"""Consumption-based material intensities for final demand, Exiobase 2015"""
# modified by Julia Sborz, on 08.05.2024, to include material categories

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
cwd = os.getcwd()
print(cwd)

C:\Users\juliasbo


In [4]:
# Bring Y, Z, A, and F matrixes
df_Y = pd.read_csv(r"X:\indecol\Projects\EXIOBASE_dev\EXIOBASE_3_8_2\upload_to_Box\private\IOT_txt\pxp\IOT_2019_pxp\Y.txt", sep='\t', header=[0,1], index_col=[0,1])
df_Z = pd.read_csv(r"X:\indecol\Projects\EXIOBASE_dev\EXIOBASE_3_8_2\upload_to_Box\private\IOT_txt\pxp\IOT_2019_pxp\Z.txt", sep='\t', header=[0,1], index_col=[0,1])
df_A = pd.read_csv(r"X:\indecol\Projects\EXIOBASE_dev\EXIOBASE_3_8_2\upload_to_Box\private\IOT_txt\pxp\IOT_2019_pxp\A.txt", sep='\t', header=[0,1], index_col=[0,1])
# from satellite accounts
df_F = pd.read_csv(r"X:\indecol\Projects\EXIOBASE_dev\EXIOBASE_3_8_2\upload_to_Box\private\IOT_txt\pxp\IOT_2019_pxp\satellite\F.txt", sep='\t', header=[0,1], index_col=[0])

FileNotFoundError: [Errno 2] No such file or directory: 'X:\\indecol\\Projects\\EXIOBASE_dev\\EXIOBASE_3_8_2\\upload_to_Box\\private\\IOT_txt\\pxp\\IOT_2019_pxp\\Y.txt'

In [24]:
# Read file with categorization into 4 material groups
new_grouping = pd.read_csv("\\\\home.ansatt.ntnu.no\\juliasbo\\Documents\\PhD_research\\Models\\2020_2019\\Data\\new_grouping_2019.csv",index_col=[0])

In [6]:
# Lists of regions, products and FD categories

regions = list(df_Z.index.get_level_values(0).unique())
products = list(df_Z.index.get_level_values(1).unique())
FD_categories = list(df_Y.columns.get_level_values(1).unique())

In [7]:
# Calculate Leontief's inverse

matrixI = np.identity(9800)
matrixImA = matrixI - df_A.values
from numpy.linalg import inv
matrixL = inv(matrixImA)

In [8]:
# Calculate xout

df_xout = df_Z.sum(axis=1) + df_Y.sum(axis=1)
array_xout = df_xout.values
print(array_xout)

[    0.           359.79773658   864.54809943 ... 35533.35519031
  6245.82863651     0.        ]


In [9]:
# Check that xout = L*(sum FD)
array_sFD = df_Y.sum(axis=1)        # it's not an array? 
xout2 = matrixL@array_sFD           # Apparently, we can multiply matrix with a pandas.series (?!)
print(xout2)                        # We get a numpy array! 

[    0.           359.79773658   864.54809943 ... 35533.35519031
  6245.82863651     0.        ]


In [10]:
xout2.sum()

157028217.75839323

In [15]:
# Calculate total domestic extraction used (sum of lines) = (raw material extraction = RME) - ORIGINAL, FROM J.O., for total MF
#sRME = np.sum(df_F.loc[df_F.index.str.contains('Domestic Extraction Used')], axis=0) #  [470:687]
#array_sRME = sRME.to_numpy()
#check1 = np.sum(array_sRME[0:])/1000
#print(check1, "Mt")

# Classify and sum the domestic extraction used (sum of lines) according to the 4 material categories - MODIFIED, FROM J.S., for MF per material type
sRME = df_F.loc[df_F.index.str.contains('Domestic Extraction Used')]
# merge new grouping to the original file
# in order to merge, both should have just one level (multi-indexed data cannot be joined or merged with simple indexed data)
sRME_columns = sRME.columns 
sRME_grouped = pd.concat([sRME,new_grouping], axis = 1)
# remove unused, if it is included in the grouping file
sRME_grouped = sRME_grouped.loc[sRME_grouped.index.str.contains('Domestic Extraction Used')]
# sum by group
sRME_grouped_sum = sRME_grouped.groupby(['new_impact']).sum()

In [81]:
# check total MF
print(sRME_grouped_sum.sum().sum()/7404000, "t/capita") # 10**6 \ 10**6, Mt/Million people = t/person, value should be the same as in the code for total MF of the same year

10.649966564277642 t/capita


In [83]:
# Calculate production-based intensities, array_PBi

with np.errstate(divide="ignore", invalid="ignore"):
    array_PBi = sRME_grouped_sum / array_xout      # These need to be in the same shape, hence above line
    array_PBi[array_PBi == np.inf] = 0      # convert inf to zeros
    array_PBi = np.nan_to_num(array_PBi)    # convert NAN to zeros
print(array_PBi)

[[0.         3.93677453 3.82905675 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [84]:
# Calculate consumption-based intensities, array_CBi
array_CBi = array_PBi@matrixL

In [102]:
# Calculate PBA and CBA domestic extraction used (Mt)
PBA_sRME = np.array(sRME_grouped_sum)/1000          # (ok, territorial accounts are basically F)
CBA_sRME = np.array(array_CBi)*np.stack((array_sFD,) * 4, axis=0)/1000 # 

In [106]:
# Check that CBA and PBA give the same total material use at the global level
print(CBA_sRME.sum().sum(), "total CBA")
print(PBA_sRME.sum().sum(), "total PBA")

78830.55291667399 total CBA
78852.35244191167 total PBA


In [120]:
# Multiply final demand with CBi (you need to use df.mul-function, otherwise a shape problem)
MF1_biomass = df_Y.mul(array_CBi[0], axis=0)        # shape (9800, 343)
MF2_biomass = MF1_biomass.groupby(["sector"]).sum()      # shape (200, 343)

MF1_fossil = df_Y.mul(array_CBi[1], axis=0)        # shape (9800, 343)
MF2_fossil = MF1_fossil.groupby(["sector"]).sum()      # shape (200, 343)

MF1_metal = df_Y.mul(array_CBi[2], axis=0)        # shape (9800, 343)
MF2_metal = MF1_metal.groupby(["sector"]).sum()      # shape (200, 343)

MF1_nonmetal = df_Y.mul(array_CBi[3], axis=0)        # shape (9800, 343)
MF2_nonmetal = MF1_nonmetal.groupby(["sector"]).sum()      # shape (200, 343)

In [121]:
# Check global material footprint (MF)
print((MF2_biomass.sum().sum() + MF2_fossil.sum().sum() + MF2_metal.sum().sum() + MF2_nonmetal.sum().sum())/1000, "Mt")

78830.55291667399 Mt


In [170]:
# CBi of FD, per category
FD2 = df_Y.groupby(["sector"]).sum()      # shape (200, 343)

# per category
CBi_FD_biomass = MF2_biomass/FD2
CBi_FD_fossil = MF2_fossil/FD2
CBi_FD_metal = MF2_metal/FD2
CBi_FD_nonmetal = MF2_nonmetal/FD2

# append them all together
CBi_FD = pd.concat([CBi_FD_biomass,CBi_FD_fossil, CBi_FD_metal, CBi_FD_nonmetal], axis = 1)

# Dataframe with only household demand, transpose
CBi_FDhh = CBi_FD.drop(["Final consumption expenditure by non-profit organisations serving households (NPISH)", "Final consumption expenditure by government", "Gross fixed capital formation", "Changes in inventories","Changes in valuables", "Exports: Total (fob)"], axis=1, level=1).T

CBi_FDhh.insert(loc=0, column='Material Category', value='NA')

CBi_FDhh.iloc[0:49,0] = "Biomass"
CBi_FDhh.iloc[50:98,0] = "Fossil Fuels"
CBi_FDhh.iloc[99:147,0] = "Metals"
CBi_FDhh.iloc[148:196,0] = "Non-metallic minerals"
CBi_FDhh = CBi_FDhh.droplevel(1, axis=0)

In [171]:
# export
CBi_FDhh.to_excel("\\\\home.ansatt.ntnu.no\\juliasbo\\Documents\\PhD_research\\Models\\2020_2019_4categories\\Output\\" + "CBi_FDhh_4MF.xlsx")

In [174]:
#select only electricity waste to use as input in the weighted average
electricity_waste = CBi_FDhh.filter(regex='Material Category|countries|electricity|Electricity|waste|landfill')
electricity_waste.head() # 33 columns, ok
#export
electricity_waste.to_excel("\\\\home.ansatt.ntnu.no\\juliasbo\\Documents\\PhD_research\\Models\\2020_2019_4categories\\Output\\" + "electricity_waste.xlsx")